## Using Reddit API : 
1. USE PYTHON praw libraries to read reddit comments 

In [ ]:
import openai
import os


In [2]:
import praw

In [3]:
os.environ['REDDIT_CLIENT'] = "Get you Reddit Client ID"
os.environ['REDDIT_CLIENT_SECRET'] = "Get you Reddit Client Secret Passcode"

In [4]:
client_id= os.environ['REDDIT_CLIENT']
client_secret=os.environ['REDDIT_CLIENT_SECRET']

In [5]:
reddit = praw.Reddit(client_id =client_id, client_secret=client_secret, user_agent='sentiment analysis test')

In [6]:
reddit

In [7]:
for submission in reddit.subreddit("stocks").hot(limit=5):
    print(submission.title)


Rate My Portfolio - r/Stocks Quarterly Thread March 2023
/r/Stocks Weekend Discussion Saturday - May 20, 2023
When did we collectively decide Google is a $126 stock after struggling at $90?
Pypl, Paypal valuation cheap
Do 401k contributions reduce the amount of unemployment you would get?


In [8]:
subreddit_stocks = reddit.subreddit("stocks")

In [9]:
subreddit_stocks.display_name

'stocks'

In [10]:
subreddit_stocks.accounts_active ## gives number of people active now 

3676

In [11]:
'''
So what we're going to do is let's say we wanted to access the first five hot posts and then those three
most upvoted comments in those posts, How do we do this?

'''

"\nSo what we're going to do is let's say we wanted to access the first five hot posts and then those three\nmost upvoted comments in those posts, How do we do this?\n\n"

In [12]:
for post in subreddit_stocks.hot(limit=5):   #first five hot posts
        print(post.title)
        submission =reddit.submission(post.id)
        #print top 2 comments per title submission
        icounter =0
        for comment in submission.comments:
                # print(comment.body)
                if comment.body=='[deleted]':
                        pass
                icounter+=1
                if icounter==2:
                        break



Rate My Portfolio - r/Stocks Quarterly Thread March 2023
/r/Stocks Weekend Discussion Saturday - May 20, 2023
When did we collectively decide Google is a $126 stock after struggling at $90?
Pypl, Paypal valuation cheap
Do 401k contributions reduce the amount of unemployment you would get?


Welcome back, everyone, to this lecture where we're actually going to generate the prompts.

So we're going to create a function that is going to get the top end post titles from some requested

subreddit and then the top end comments for each post.

And we'll let the user decide on how many post titles they're interested in, as well as how many comments

OPEN API - Prompt generation 

In [13]:
import os
import openai

In [26]:
os.environ["OPENAI_API_KEY"] = "Get you OPENAI key"  
openai.api_key = os.getenv("OPENAI_API_KEY")

Genearte Prompt

In [15]:
getattr(subreddit_stocks, "hot")

<bound method BaseListingMixin.hot of Subreddit(display_name='stocks')>

In [16]:
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=6, num_comments=3, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for icounter, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):

        if icounter < skip_first : 
            continue

        icounter+=(1-skip_first)
        titles_and_comments[icounter] = ""
        submission = reddit.submission(post.id)
        title = post.title

        titles_and_comments[icounter] += "Title: " + title  + "\n\n"
        titles_and_comments[icounter] += "Comments: \n\n"

        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[icounter] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break
    return titles_and_comments
    

In [17]:
titles_and_comments = get_titles_and_comments(subreddit="stocks", limit=12)

In [18]:
titles_and_comments

{1: 'Title: When did we collectively decide Google is a $126 stock after struggling at $90?\n\nComments: \n\nGoogle is hardly the crazy one. Nvidia was also around 90 and now is over 300.\nAi, Ai, Ai, Ai, AI, AI, AIiiii\nImagine thinking anyone here has any effect on a ticker like google\n',
 2: "Title: Pypl, Paypal valuation cheap\n\nComments: \n\nIt's hard to make a strong bullish case for PYPL.\n\nLong term, competition abounds.  Their current CEO made a fool of himself saying it was undervalued 200% higher, and now he's taking a golden parachute exit this year.  \n\nBut a more mild bull argument would mention that Venmo has a solid presence, that PayPal does well in Europe, and earnings have been... decent. \n\nNow some intangibles that could give up some unexpected and perhaps undeserved upside.  One is that falling from 300+ to $59 may mean it's been significantly derisked.  Another is that the market should reward them when the current failed CEO gets replaced.  \n\nAnother big 

In [19]:
print(titles_and_comments)

{1: 'Title: When did we collectively decide Google is a $126 stock after struggling at $90?\n\nComments: \n\nGoogle is hardly the crazy one. Nvidia was also around 90 and now is over 300.\nAi, Ai, Ai, Ai, AI, AI, AIiiii\nImagine thinking anyone here has any effect on a ticker like google\n', 2: "Title: Pypl, Paypal valuation cheap\n\nComments: \n\nIt's hard to make a strong bullish case for PYPL.\n\nLong term, competition abounds.  Their current CEO made a fool of himself saying it was undervalued 200% higher, and now he's taking a golden parachute exit this year.  \n\nBut a more mild bull argument would mention that Venmo has a solid presence, that PayPal does well in Europe, and earnings have been... decent. \n\nNow some intangibles that could give up some unexpected and perhaps undeserved upside.  One is that falling from 300+ to $59 may mean it's been significantly derisked.  Another is that the market should reward them when the current failed CEO gets replaced.  \n\nAnother big o

In [20]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments

In [21]:
prompt = create_prompt(titles_and_comments[1])

In [22]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: When did we collectively decide Google is a $126 stock after struggling at $90?

Comments: 

Google is hardly the crazy one. Nvidia was also around 90 and now is over 300.
Ai, Ai, Ai, Ai, AI, AI, AIiiii
Imagine thinking anyone here has any effect on a ticker like google



In [27]:
for key, title_and_comments in titles_and_comments.items():
    prompt = create_prompt(title_and_comments)

    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment: " + response["choices"][0]["text"])
    print("-"*30)

Title: When did we collectively decide Google is a $126 stock after struggling at $90?

Comments: 

Google is hardly the crazy one. Nvidia was also around 90 and now is over 300.
Ai, Ai, Ai, Ai, AI, AI, AIiiii
Imagine thinking anyone here has any effect on a ticker like google

Sentiment: 
Company: Google
Sentiment: Neutral
------------------------------
Title: Pypl, Paypal valuation cheap

Comments: 

It's hard to make a strong bullish case for PYPL.

Long term, competition abounds.  Their current CEO made a fool of himself saying it was undervalued 200% higher, and now he's taking a golden parachute exit this year.  

But a more mild bull argument would mention that Venmo has a solid presence, that PayPal does well in Europe, and earnings have been... decent. 

Now some intangibles that could give up some unexpected and perhaps undeserved upside.  One is that falling from 300+ to $59 may mean it's been significantly derisked.  Another is that the market should reward them when the cu